In [1]:
import torch
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
from torch import nn

import cv2
import numpy as np
import copy
import time
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import sys
sys.path.append("../")
from utilities import helper_functions, splitting, augmentations, metrics

## Define Dataset

In [2]:
class MultiViewDataset(Dataset):
    '''
    Brands Dataset
    '''
    
    def __init__(self, img_paths, labels, augmentation_function, png=True):
        '''
        Initialize the dataset
        '''
        assert len(img_paths) == len(labels), "Number of files should match number of targets"
        
        self.img_files = img_paths
        self.labels = labels
        self.augmentation_function = augmentation_function
        self.png = png
    
    def load_png(self, img_path):
        '''
        This function loads an image from a PNG path
        '''
        try:
            actual_image = cv2.imread(img_path)
        except:
            print(f"Something went wrong with reading file {img_path}")
            actual_image = np.zeros((256,256)).astype(np.uint8)
        
        actual_image = Image.fromarray(actual_image)
        
        return actual_image
    
    
    def load_dicom(self, img_path):
        '''
        This function loads an image from a DICOM path
        '''
        try:
            image_info = pydicom.dcmread(img_path)
            actual_image = image_info.pixel_array
            
        except:
            print(f"Something went wrong with reading file {img_path}")
            actual_image = np.zeros((256,256))
        
        actual_image = helper_functions.prepare_image(actual_image, rgb=True, channels_first=True)
        actual_image = Image.fromarray(actual_image)

        return actual_image
    
    def handle_views(self, view):
        '''
        Function to return appropriate array based on view
        '''
        if view == "AP":
            return np.ones((1,))
        elif view == "L":
            return np.ones((1, )) + 1
    
    def __getitem__(self, index):
        '''
        Get a unique item from the dataset according to index. This is required when building a custom dataloader
        '''
        img_row = self.img_files.iloc[index,:]
        if self.png:
            X_image = self.load_png(img_row.filepath)
        else:
            X_image = self.load_dicom(img_row.filepath)
        if self.augmentation_function is not None:
            X_image = self.augmentation_function(X_image)
        X_view = self.handle_views(img_row.View)
        Y = self.labels[index]
        return (X_image.float(), torch.as_tensor(X_view).float()), torch.as_tensor(Y).long()
        
    
    def __len__(self):
        '''
        Length of the dataset. This is required when building a custom dataloader
        '''
        return len(self.img_files)                

## Define BasicMultiView Model

In [3]:
class BasicMV_AV(nn.Module):
    '''
    Pretrained model backbone, two linear layers with dropout in between.
    View feature appended after FC of pretrained network
    '''
    def __init__(self, pretrained, in_features, num_classes, dropout_1 = 0.15, dropout_2=0.5): 
        '''
        Initializing 2 linear layers, dropout and Leaky RelU layers
        '''
        super(BasicMV_AV, self).__init__()
        self.backbone = pretrained
        self.fc1 = nn.Linear(in_features, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.dropout_1 = nn.Dropout(dropout_1)
        self.dropout_2 = nn.Dropout(dropout_2)
        self.activation = nn.LeakyReLU()

    def forward(self, x): 
        '''
        Forward pass through network
        '''
        out = self.dropout_1(self.activation(self.backbone(x[0])))
        out = torch.cat([out, x[1]], dim=-1)
        out = self.activation(self.fc1(out))
        out = self.fc2(self.dropout_2(out))

        return out

## Define Training Function
Adapted from https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [4]:
def train_bmv_model(model, criterion, optimizer, scheduler, num_epochs, dataloaders, dataset_sizes, device):
    '''
    Function to train a simple PyTorch model.
    It returns the model with the best validation accuracy (we might change this metric)
    Returns losses and accuracies too
    '''
    LOSSES = {'train':[], 'val':[]}
    ACCS = {'train':[], 'val':[]}
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in tqdm.notebook.tqdm(range(num_epochs)):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode        
   
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data.
            for inputs, labels in tqdm.notebook.tqdm(dataloaders[phase]):
                inputs = list(map(lambda x: x.to(device), inputs))
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs[0].size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            LOSSES[phase].append(epoch_loss)
            ACCS[phase].append(epoch_acc.cpu().item())

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, LOSSES, ACCS

## Define Evaluation Function

In [5]:
def evaluate_model(model, dataloaders, device, phase='test'):
    '''
    Function to evaluate on test set
    '''
    model.eval()   # Set model to evaluate mode        

    running_loss = 0.0
    running_corrects = 0
    running_incorrects = 0
    pred_list = []
    label_list = []
    output_list = []

    # Iterate over data.
    for inputs, labels in tqdm.notebook.tqdm(dataloaders[phase]):
        inputs = list(map(lambda x: x.to(device), inputs))
        labels = labels.to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        # statistics
        running_corrects += torch.sum(preds == labels.data)
        running_incorrects += torch.sum(preds != labels.data)
        pred_list.extend(preds.cpu().tolist())
        output_list.extend(outputs.cpu().tolist())
        label_list.extend(labels.cpu().tolist())
        
    print('Total Correct Predictions: ' + str(running_corrects))
    print('Total Incorrect Predictions: ' + str(running_incorrects))
    
    return np.array(pred_list), nn.Softmax(dim=-1)(torch.Tensor(output_list)).numpy(), np.array(label_list)

## Workflow for One Iteration

In [6]:
def workflow(data, data_to_drop, preprocessed, pretrained, dropout_1, dropout_2, epochs):
    # Parameters
    DROPOUT_1 = dropout_1
    DROPOUT_2 = dropout_2
    EPOCHS = epochs
    
    print("Prepping and Splitting Data...")
    # Prepping the dataset, removing files to remove
    dataset = data.copy()
    dataset['filepath']  = "../" + dataset['filepath'].str[14:]
    dataset = dataset[~dataset['filepath'].isin(data_to_drop)].reset_index(drop=True)
    
    # If we want the preprocessed images then modify the filepaths
    if preprocessed:
        dataset['filepath'] = dataset['filepath'].str[:-4] + "_processed.png"
    
    # Split into train, val and test
    train, val, test = splitting.split_data_2(dataset, "anterior", return_data=True, save_data = False)
    
    
    #################################################
    # GETTING MEANS AND STDS OF TRAIN DATA FOR NORM #
    #################################################
    # Augmentation without norm function
    print("Getting Means and Standard Deviation of Train Set...")
    aug_wo_norm = augmentations.apply_augmentations_wo_norm(data_set="train", size_val=256, contrast_val=0.25, 
                                                        hue_val=0.25, grayscale_prob=0.1,
                                                        hflip_prob=0.5, degree_val=0, 
                                                        shear_val=5)
    # Creating train dataset
    train_dataset = MultiViewDataset(img_paths=train[['filepath','View']], labels=train['Label'], 
                                     augmentation_function=aug_wo_norm, png=True if preprocessed else False)
    # Getting means and stds
    dataset_means, dataset_stds = helper_functions.get_training_mean_std_bmv(train_dataset)
    
    
    
    #########################
    # TRAINING ACTUAL MODEL #
    #########################    
    print("Getting Dataloaders, Parameters and Models Ready...")
    # Augmentation functions with normalization
    param_aug = {'size_val': 256, 'contrast_val':0.25, 
                 'hue_val': 0.25, 'grayscale_prob': 0.1,
                 'hflip_prob': 0.5, 'degree_val':0,
                 'shear_val':5, 'avg_pop_mean': dataset_means, 
                 'avg_pop_std': dataset_stds
    }
    aug_functions = {x: augmentations.apply_augmentations_with_norm(data_set=x, **param_aug) for x in ['train', 'val', 'test']}
    
    # Datasets and Dataloaders
    datasets = {
        'train': MultiViewDataset(img_paths=train[['filepath','View']], labels=train['Label'], 
                                  augmentation_function=aug_functions['train'], png=True if preprocessed else False),
        'val': MultiViewDataset(img_paths=val[['filepath','View']], labels=val['Label'], 
                                  augmentation_function=aug_functions['val'], png=True if preprocessed else False), 
        'test':  MultiViewDataset(img_paths=test[['filepath','View']], labels=test['Label'], 
                                  augmentation_function=aug_functions['test'], png=True if preprocessed else False)
    }

    params = {'batch_size': 8, 'shuffle': True, 'num_workers': 1}

    dataloaders_ = {x: DataLoader(datasets[x], **params) for x in ['train', 'val']}
    dataloaders_['test'] = DataLoader(datasets['test'], batch_size=1, shuffle=False, num_workers=1)

    dataset_sizes_ = {x: len(datasets[x]) for x in ['train', 'val', 'test']}
    
    # Other parameters
    num_classes = len(train['Label'].value_counts())
    
    # Weights
    value_counts_ = train['Label'].value_counts().sort_index()
    weights = torch.Tensor(value_counts_.sum()/(len(value_counts_)*value_counts_).tolist())
    
    # Selecting device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # Creating models
    if pretrained == "resnet34":
        backbone_model = models.resnet34(pretrained=True)
    elif pretrained == "resnet50":
        backbone_model = models.resnet50(pretrained=True)
    elif pretrained == "densenet121":
        backbone_model = models.densenet121(pretrained=True)
    
    basicmvav_model = BasicMV_AV(backbone_model, 1001, num_classes, dropout_1=DROPOUT_1, dropout_2=DROPOUT_2)
    basicmvav_model = basicmvav_model.to(device)
    
    # Criterion
    criterion_ = torch.nn.CrossEntropyLoss(weight=weights)
    criterion_ = criterion_.to(device)

    # optimizer
    optimizer_ = torch.optim.Adam([
        {'params': list(basicmvav_model.parameters())[:-6], 'lr': 0.00021},
        {'params': list(basicmvav_model.parameters())[-6:], 'lr': 0.00063}
    ])
    
    # LR Decay
    exp_lr_scheduler_ = lr_scheduler.StepLR(optimizer_, step_size=3, gamma=0.5)
    
    print("Training and Evaluating Model...")
    # Training model
    basicmvav_model, LOSSES, ACCS = train_bmv_model(model=basicmvav_model, criterion=criterion_, 
                                                   optimizer=optimizer_, scheduler=exp_lr_scheduler_,
                                                   num_epochs=EPOCHS, dataloaders=dataloaders_, 
                                                   dataset_sizes=dataset_sizes_, device = device)
    
    # Evaluating model
    preds, probs, labels = evaluate_model(basicmvav_model, dataloaders=dataloaders_, device = device)
    
    # Metrics
    f1_score, precision, recall, auc, confusion_matrix = helper_functions.metrics_function(preds, probs, labels)
    
    # move everything off the cuda to prevent crashing it
    basicmvav_model = basicmvav_model.cpu()
    criterion_ = criterion_.cpu()
    
    return LOSSES, ACCS, f1_score, precision, recall, auc, confusion_matrix

## Tuning Function
For multiple iterations

In [7]:
def multiple_its(data, suffix, data_to_drop, preprocessed, pretrains, dropouts_1, dropouts_2, epochs, samples_per_model):
    '''
    Grid Search parameters
    '''
    gridsearch_params = [
    (pretrain, dropout_1, dropout_2)
    for pretrain in pretrains
    for dropout_1 in dropouts_1
    for dropout_2 in dropouts_2
    ]
    
    # For each hyperparameter combination
    for pretrain, dropout_1, dropout_2 in gridsearch_params:
        file_name = f"results/bmav/tuning/{pretrain}-dropout1{dropout_1}-dropout2{dropout_2}-preprocessed{preprocessed}-{suffix}"
        
        f1_scores = []
        precisions = []
        recalls = []
        aucs = []
        
        # Loop through and run n iterations
        for i in tqdm.notebook.tqdm(range(samples_per_model)):
            print(f'''
MODEL:\n
Pretrain: {pretrain} | Dropout1: {dropout_1} | Dropout2: {dropout_2}\n
Split {i}
            ''')
            # Compute results from one workflow
            LOSSES, ACCS, f1_score, precision, recall, auc, confusion_matrix = workflow(data=data, 
                                                                                        data_to_drop=data_to_drop, 
                                                                                        preprocessed=preprocessed, 
                                                                                        pretrained = pretrain,
                                                                                        dropout_1=dropout_1, 
                                                                                        dropout_2=dropout_2, 
                                                                                        epochs=epochs)
            
            # Append those results
            f1_scores.append(f1_score)
            precisions.append(precision)
            recalls.append(recall)
            aucs.append(auc)
        
        # Compile the results
        print(f"Compiling Results from {samples_per_model} splits...")
        compiled_numeric = metrics.compile_numeric_results(f1_scores, precisions, recalls, aucs)
        compiled_numeric.index.name = 'Score'
        
        compiled_numeric.to_csv(file_name + "_numeric.csv", index=True)
        
        # This part is a bit buggy sometimes so I removed it
#     best_params = 0
#     f1 = 0
#     for pretrain, dropout_1, dropout_2 in gridsearch_params:
#         f1_read = pd.read_csv(f"results/bmav/aug/{pretrain}-dropout1{dropout_1}-dropout2{dropout_2}-preprocessed{preprocessed}-{suffix}_numeric.csv").iloc[0,1]

#         if f1_read > f1:
#             f1 = f1_read
#             best_params = (pretrain, dropout_1, dropout_2)
    
#     print("Best Parameters:", best_params)

## Tuning

In [8]:
data_ = pd.read_csv("../Master_Anterior_HDW.csv")
suffix_ = "anterior"
data_to_drop_ = []
preprocessed_ = True
pretrains_ = ['densenet121'] 
dropouts_1_ = [0.1]
dropouts_2_ = [0.2]
epochs_ = 20
samples_per_model_ = 3

In [9]:
multiple_its(data=data_, suffix=suffix_, data_to_drop=data_to_drop_, preprocessed=preprocessed_, 
             pretrains=pretrains_, dropouts_1=dropouts_1_, dropouts_2=dropouts_2_, epochs=epochs_, 
             samples_per_model=samples_per_model_)


MODEL:

Pretrain: densenet121 | Dropout1: 0.1 | Dropout2: 0.2

Split 0
            
Prepping and Splitting Data...
Getting Means and Standard Deviation of Train Set...
Getting Dataloaders, Parameters and Models Ready...
Training and Evaluating Model...


Epoch 0/19
----------



train Loss: 1.4259 Acc: 0.4287



val Loss: 1.2956 Acc: 0.4430

Epoch 1/19
----------



train Loss: 1.2886 Acc: 0.4184


...


train Loss: 0.0090 Acc: 0.9968



val Loss: 2.6498 Acc: 0.7824

Epoch 19/19
----------



train Loss: 0.0037 Acc: 1.0000



val Loss: 2.8761 Acc: 0.7977


Training complete in 16m 8s
Best val Acc: 0.801527



Total Correct Predictions: tensor(384, device='cuda:0')
Total Incorrect Predictions: tensor(71, device='cuda:0')

Compiling Results from 3 splits...
